In [1]:
import stoploss as sl



In [2]:
pf = sl.Portfolio(value = 100, buying_power = 100)
strat = sl.Strategy(0.02)

ent = sl.Entry(8.14, 8.00)
pos = sl.Position(ent, strat, pf)
print(pos)

Buy 9 shares at $8.14 worth $73.26 with stop at $7.92
Risking $1.98 for 0.02 percent of a $100.00 portfolio

Price target = $8.58 (+1.05)

